In [1]:
from config import get_config

cfg = get_config()

cfg["batch_size"] = 128
cfg["preload"] = None
cfg["num_epochs"] = 30
from training import train_model

In [3]:
train_model(cfg)

Using device: cuda
